In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.applications import VGG16
from keras.optimizers import Adam
from tensorflow import keras
import random
from PIL import Image

In [2]:
model_Mid = keras.models.load_model("D:\\Project Himachal\\Data\ForCNN\\Takrimodel")
model_Top = keras.models.load_model("D:\\Project Himachal\\Data\ForCNN\\TakrimodelTop")
model_Bottom =keras.models.load_model("D:\\Project Himachal\\Data\ForCNN\\TakrimodelBottom")

master_label_Mid = ['100',  '101',  '102',  '103',  '105',  '106',  '107',  '108',  '110',  '111',  '112',  '113',  '114',  '115',  '116',  '117',  '118',  '119',  '120',  '121',  '123',  '124',  '125',  '126',  '127',  '128',  '130',  '131',  '200',  '201', '202',  '203',  '204',  '205',  '206',  '302',  '303',  '402',  '403',  '404',  '405',  '406',  '408',  '409',  '410',  '500']
master_label_Top = ['301',  '302',  '306',  '307',  '308',  '309',  '310']
master_label_Bottom = ['304',  '305',  '501',  '502',  '503']
dict_ = {}
test_df = pd.read_csv('D:\\Project Himachal\\Data\\LabelMapper.csv')
for row in test_df.iterrows():
    label = (row[1][0])
    unicode = (row[1][1])
    dict_[label] = unicode

In [3]:
def isEmpty(img_path):
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE) # reading the image
    
    hieght = image.shape[0]
    width = image.shape[1]    
    
    for y_axis in range (hieght):        
        for x_axis in range (width):
            if (image[y_axis,x_axis]!=0):            
                return 0           
    return 1

def findCharacterCNN(imges_test,location):
    #imges_test = "D:\\Project Himachal\\Data\\Temp\\Temp_Resized_CharPart.png"
    if(location=="M"):
        model = model_Mid
        master_label = master_label_Mid
    if (location=="T"):
        model = model_Top
        master_label = master_label_Top
    
    if (location=="B"):
        model = model_Bottom
        master_label = master_label_Bottom
        
    X_tst = []
    Test_imgs = []
    
    temp = cv2.imread(imges_test)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
    temp = np.expand_dims(temp, axis=2)
    X_tst.append(temp)      
    Test_imgs.append(imges_test)
    X_tst = np.asarray(X_tst)
    X_tst = X_tst.astype('float32')
    X_tst /= 255
    
    test_predictions = model.predict(X_tst)
    for answer in test_predictions:
        index = -1
        max_index = -1
        tc = 0 
        for c,v in enumerate(answer):
            if v>max_index:
                max_index = v
                index = c
            tc = tc + v
            
    label = int(master_label[index])
    #print(label)
    #print(dict_)
    return dict_[label]

def find_hieght(img_path):
    image = cv2.imread(img_path) # reading the image    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    
    hieght = binary.shape[0]
    width = binary.shape[1]
 
    ht = 0
    
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black>0):
            ht = ht+1
    
    return ht

In [4]:
def find_biggestbox(binary):
    hieght = binary.shape[0]
    width = binary.shape[1]
    buffer = 2
    x_start=0
    found=0
    for x_axis in range (width):
        black = 0
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            x_start = x_axis
        else:
            found=1
    x_start = x_start - buffer
    if (x_start<0):
        x_start = 0
    
    x_end=0
    found=0
    for x_axis in range (width):
        black = 0
        x_axis = width - x_axis-1
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            x_end = x_axis
        else:
            found=1
    if(x_end==0):
        x_end = width
    x_end = x_end + buffer
    if (x_end>width):
        x_end = width
            
    y_start=0
    found=0
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            y_start = y_axis
        else:
            found=1
    y_start = y_start - buffer
    if (y_start<0):
        y_start = 0

            
    y_end=0
    found=0
    for y_axis in range (hieght):
        black = 0
        y_axis = hieght - y_axis-1
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        if (black==0) and (found==0):
            y_end = y_axis
        else:
            found=1
    if(y_end==0):
        y_end = hieght
    y_end = y_end + buffer
    if (y_end>hieght):
        y_end = hieght
 

    return (x_start,x_end,y_start,y_end)   

In [5]:
def extract_Character_Full(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    #print(binary.shape)
    hieght = binary.shape[0]
    width = binary.shape[1]
    check = 0
    x1 = []
    y1 = []
    for x_axis in range (width):
        black = 0
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):
                weight = abs(y_axis - (hieght/2))/(hieght/2)
                if(y_axis>hieght*.7):
                    weight = weight*.4
                black = abs(black + 1 - weight)
            #print(binary[y_axis,x_axis])
        #print(black) 
        x1.append(x_axis)
        y1.append(black/hieght)

    length_ = len(y1)
    minima = []
    for index in range(length_):
        range_ = 5
        flag = 1
        for internal_loop in range (range_):
            if (index-internal_loop)>=0 and y1[index-internal_loop]<y1[index]:
                flag = 0
            if (index+internal_loop)<length_ and y1[index+internal_loop]<y1[index]:
                flag = 0
        if (flag==1) and y1[index]<.04:   
            #print(y1[index])
            minima.append(index)

    for num in minima:
        for a in range(1,5):
            if (num+a) in minima:
                minima.remove(num+a)
            else:
                break

    if 0 not in (minima):
        minima.append(0)

    if width not in (minima):
        minima.append(width)
    
    minima_v = minima

    #Now lets divide horizontally
    x1 = []
    y1 = []
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):
                weight = abs(x_axis - (width/2))/(width/2)
                black = abs(black + 1 - weight)
                #black = black + 1
            #print(binary[y_axis,x_axis])
        x1.append(y_axis)
        y1.append(black/width) 

    length_ = len(y1)
    minima = []
    for index in range(length_):
        range_ = 8
        flag = 1
        for internal_loop in range (range_):
            if (index-internal_loop)>=0 and y1[index-internal_loop]<y1[index]:
                flag = 0
            if (index+internal_loop)<length_ and y1[index+internal_loop]<y1[index]:
                flag = 0
        if (flag==1) or y1[index]<.2:
            minima.append(index)

    for num in minima:
        for a in range(1,10):
            if (num>hieght/2):
                check = num+a
            if (num<hieght/2):
                check = num-a
            if (check) in minima:
                minima.remove(check)
            else:
                break
    top_= 0
    bottom_ = hieght
    for number in minima:
        if (number > top_) and number < hieght/3:
            top_ = number
        if (number < bottom_) and (hieght-number) < hieght/3:
            bottom_ = number
    
    w = img.shape[1]
    h = img.shape[0]

    out_W=[]
    
    prev = 0
    for ind_val in minima_v:        
        if ind_val == 0:
            continue
        #print(0,top_,prev,ind_val)
        #print(top_,bottom_,prev,ind_val)
        #print(bottom_,h,prev,ind_val)
        roi0 = img[0:h,prev:ind_val]        
        out_W.append(roi0)
        prev = ind_val

    return out_W

In [6]:
def removeBottomLine(img_path):
    image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE) # reading the image
    
    hieght = image.shape[0]
    width = image.shape[1]
    
    img = np.zeros(shape=(hieght, width))
    
    val = 0
    flag = 0
    for y_axis in range (hieght):        
        val = 0
        for x_axis in range (width):
            if (image[y_axis,x_axis]==0):            
                val = val + 1                
        #print(val)
        if (val>22)or(flag==1): 
            flag = 1
            for y_axis_delta in range(2):
                for x_axis in range (width):
                    img[y_axis-y_axis_delta,x_axis] = 255                
        else:
            for x_axis in range (width):
                img[y_axis,x_axis] = image[y_axis,x_axis]
        if (val==0):
            flag=0
    return img

def make_binary(Img_Original):
    h,b = (Img_Original.shape)

    binary_img = np.zeros([h, b])

    for col in range(h):
        for row in range(b):
            if (Img_Original[col][row])>=200:
                binary_img[col][row]= 0
            else:
                binary_img[col][row]= 1

    return binary_img

def invert(Img_Original):
    h,b = (Img_Original.shape)

    binary_img = np.zeros([h, b])

    for col in range(h):
        for row in range(b):
            if (Img_Original[col][row])==0:
                binary_img[col][row]= 255
            else:
                binary_img[col][row]= 0

    return binary_img

def neighbours(x,y,image):
    "Return 8-neighbours of image point P1(x,y), in a clockwise order"
    img = image
    x_1, y_1, x1, y1 = x-1, y-1, x+1, y+1
    return [ img[x_1][y], img[x_1][y1], img[x][y1], img[x1][y1],     # P2,P3,P4,P5
                img[x1][y], img[x1][y_1], img[x][y_1], img[x_1][y_1] ]    # P6,P7,P8,P9

def transitions(neighbours):
    "No. of 0,1 patterns (transitions from 0 to 1) in the ordered sequence"
    n = neighbours + neighbours[0:1]      # P2, P3, ... , P8, P9, P2
    return sum( (n1, n2) == (0, 1) for n1, n2 in zip(n, n[1:]) )  # (P2,P3), (P3,P4), ... , (P8,P9), (P9,P2)

def thinning(image):
    "the Zhang-Suen Thinning Algorithm"
    Image_Thinned = image.copy()  # deepcopy to protect the original image
    changing1 = changing2 = 1        #  the points to be removed (set as 0)
    while changing1 or changing2:   #  iterates until no further changes occur in the image
        # Step 1
        changing1 = []
        rows, columns = Image_Thinned.shape               # x for rows, y for columns
        for x in range(1, rows - 1):                     # No. of  rows
            for y in range(1, columns - 1):            # No. of columns
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1     and    # Condition 0: Point P1 in the object regions 
                    2 <= sum(n) <= 6   and    # Condition 1: 2<= N(P1) <= 6
                    transitions(n) == 1 and    # Condition 2: S(P1)=1  
                    P2 * P4 * P6 == 0  and    # Condition 3   
                    P4 * P6 * P8 == 0):         # Condition 4
                    changing1.append((x,y))
        for x, y in changing1: 
            Image_Thinned[x][y] = 0
        # Step 2
        changing2 = []
        for x in range(1, rows - 1):
            for y in range(1, columns - 1):
                P2,P3,P4,P5,P6,P7,P8,P9 = n = neighbours(x, y, Image_Thinned)
                if (Image_Thinned[x][y] == 1   and        # Condition 0
                    2 <= sum(n) <= 6  and       # Condition 1
                    transitions(n) == 1 and      # Condition 2
                    P2 * P4 * P8 == 0 and       # Condition 3
                    P2 * P6 * P8 == 0):            # Condition 4
                    changing2.append((x,y))    
        for x, y in changing2: 
            Image_Thinned[x][y] = 0    
    return Image_Thinned
 
def imshow_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    cv2.imshow('labeled.png', labeled_img)
    cv2.waitKey()
    
def imsave_components(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179*labels/np.max(labels))
    blank_ch = 255*np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    cv2.imshow('labeled.png', labeled_img)

def find_index(arr):
    index = 0
    row = 0
    flag=0
    for l in arr:
        #print(l,np.sum(l))
        if (np.sum(l)==0) and (flag==0):
            index=row
        else:
            flag = 1

        row=row+1
    return index

def keep_max(labels_im):
    b = labels_im.flatten()
    counter = collections.Counter(b)
    input_arr1 = np.copy(labels_im)
    
    max = 0
    for c in counter:
        if c==0:
            continue
        if (counter[c]>max):
            out = c
            max = counter[c]
    input_arr1[input_arr1!=out] = 0
    input_arr1[input_arr1==out] = 1
    return input_arr1

def keep_max2(labels_im):
    hieght = labels_im.shape[0]
    width = labels_im.shape[1]
    
    singular = []
    for y_axis in range (hieght):
        val = 0
        for x_axis in range (width):
            if (labels_im[y_axis,x_axis]!=0):            
                val = labels_im[y_axis,x_axis]
        
        singular.append(val)

    #b = singular.flatten()
    counter = collections.Counter(singular)
    input_arr1 = np.copy(labels_im)
    
    max = 0
    out = 0
    for c in counter:
        if c==0:
            continue
        if (counter[c]>max):
            out = c
            max = counter[c]
    
    input_arr1[input_arr1!=out] = 0
    input_arr1[input_arr1==out] = 1
    return input_arr1

def filled_ratio(img_path):
    image = cv2.imread(img_path) # reading the image
    image_copy = cv2.imread(img_path) # reading the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary

    hieght = binary.shape[0]
    width = binary.shape[1]
    
    val = 0
    for y_axis in range (hieght):        
        for x_axis in range (width):
            if (binary[y_axis,x_axis]!=0):            
                val = val + 1
    
    return val/(hieght*width)

In [7]:
def makeSameSize(img_path,savePath):
    img_b = Image.open("D:\\Project Himachal\\Data\\Labelled\\background.png")
    img_o = Image.open(img_path)
    img = cv2.imread(img_path)            
    w = (img.shape[1])
    h = (img.shape[0])
    img_b.paste(img_o, (int((50-w)/2),int((60-h)/2)), mask = img_o)
    img_b.save(savePath)

In [8]:

def divide_page_into_lines(img_path):
    image = cv2.imread(img_path) # reading the image
    image_copy = cv2.imread(img_path) # reading the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    #cv2.imshow('binary', binary)
    cv2.imwrite('D:\\Project Himachal\\Data\\Temp\\Page_binary.png', binary)
    hieght = binary.shape[0]
    width = binary.shape[1]
    
    x1 = []
    y1 = []
    for y_axis in range (hieght):
        black = 0
        for x_axis in range (width):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        x1.append(y_axis)
        y1.append(black)
        
    minima = 100
    flag = "D"
    lineBreaks = []
    for idx, item in enumerate(y1):
        #print(idx,item)

        if(item<minima):
            minima=item
            flag = "D"

        if(item>minima) and (flag=="D"):
            minima=item
            flag = "I"
            #print(idx-3)
            lineBreaks.append(idx-3)

    lineBreaks.append(hieght-3) 
    
    line_breaks = []
    size = len(lineBreaks)
    cc=0
    while cc<size-1:
        #print(lineBreaks[index])
        if (lineBreaks[cc+1]-lineBreaks[cc])>15:
            line_breaks.append(lineBreaks[cc])
            cc=cc+1
        else:
            line_breaks.append(lineBreaks[cc+1])
            cc = cc+2

    lineBreaks = line_breaks

    line_breaks = []
    size = len(lineBreaks)
    cc=0
    while cc<size-1:
        #print(lineBreaks[index])
        if (lineBreaks[cc+1]-lineBreaks[cc])>15:
            line_breaks.append(lineBreaks[cc])
            cc=cc+1
        else:
            line_breaks.append(lineBreaks[cc+1])
            cc = cc+2

    lineBreaks = line_breaks
    
    finalOutput = []
    b_=0
    for c,b in enumerate(lineBreaks[1:]):
        #print("Processing Line " + str(c))
        #print("Processing Line " + str(b_),str(b))
        roi=image[b_:b,0:width]
        cv2.imwrite('D:\\Project Himachal\\Data\\Temp\\Temp_Line.png', roi)
        #print(filled_ratio('D:\\Project Himachal\\Data\\Temp\\Temp_Line.png'))
        out = divide_line_into_words('D:\\Project Himachal\\Data\\Temp\\Temp_Line.png')
        finalOutput.append(out)
        b_=b
    
    return finalOutput

In [9]:
def divide_line_into_words(img_path):
    image = cv2.imread(img_path) # reading the image
    image_copy = cv2.imread(img_path) # reading the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
    #cv2.imshow('binary', binary)
    cv2.imwrite('D:\\Project Himachal\\Data\\Temp\\L_binary.png', binary)
    
    hieght = binary.shape[0]
    width = binary.shape[1]
 
    x1 = []
    y1 = []
    for x_axis in range (width):
        black = 0
        for y_axis in range (hieght):
            if (binary[y_axis,x_axis]==0):            
                black = black+1
            #print(binary[y_axis,x_axis])
        #print(black) 
        x1.append(x_axis)
        y1.append(black)
        
    minima = 100
    min_count=0
    flag = "D"
    lineBreaks = []
    for idx, item in enumerate(y1):
        #print(idx,item)

        if(item<minima):
            minima=item
            flag = "D"

        if(item>minima):        
            minima=item
            if (flag=="D") and (p_item==0):
                lineBreaks.append(idx-3)   
            flag = "I"
            #print(idx-3)        

        p_item = 0
        for p in range(5):
            p_item = p_item + y1[idx-p]

    lineBreaks.append(width-3)
    
    finalOutput = []
    
    b_=0
    for c,b in enumerate(lineBreaks[1:]):
        #print("Processing Word " + str(c))
        roi=image[0:hieght,b_:b]  
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY) # convert2grayscale
        (thresh, binary) = cv2.threshold(gray, 150,255, cv2.THRESH_BINARY) # convert2binary
        xs,xe,ys,ye = find_biggestbox(binary)   
        if (ye-ys) < 10:
            continue
        cv2.imwrite('D:\\Project Himachal\\Data\\Temp\\Temp_Word.png', binary[ys:ye,xs:xe]  )
        if (filled_ratio('D:\\Project Himachal\\Data\\Temp\\Temp_Word.png')>.95):
            continue
        out = divide_word_into_characters('D:\\Project Himachal\\Data\\Temp\\Temp_Word.png')
        finalOutput.append(out)
        b_=b
        
    return finalOutput
        

In [10]:
def divide_word_into_characters(img_path):
    image = cv2.imread(img_path) # reading the image
    image_copy = cv2.imread(img_path) # reading the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
    (thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary

    out = extract_Character_Full(image)
    finalOutput = []
    
    for idx,val in enumerate(out):
        #print("Processing Character " + str(idx))
        cv2.imwrite('D:\\Project Himachal\\Data\\Temp\\Temp_Character.png', val  )
        if (filled_ratio('D:\\Project Himachal\\Data\\Temp\\Temp_Character.png')>0.95):
            continue
        
        output = divide_character_into_parts('D:\\Project Himachal\\Data\\Temp\\Temp_Character.png')
        finalOutput.append(output)
        
    return finalOutput
    
def divide_character_into_parts(img_path):
    out = []
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    bin_img = make_binary(img)
    bw_Skeleton = thinning(bin_img)  
    plt.imsave("D:\\Project Himachal\\Data\\Temp\\Temp_Skeleton.png",(bw_Skeleton), cmap=plt.cm.gray )
    
    img_new = cv2.imread('D:\\Project Himachal\\Data\\Temp\\Temp_Skeleton.png', 0)
    img_new = cv2.threshold(img_new, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
    num_labels, labels_im = cv2.connectedComponents(img_new)

    input_arr1 = keep_max2(labels_im)

    start = (find_index(input_arr1))
    end = (find_index(input_arr1[::-1]))
    end_ = input_arr1.shape[0] - end

    #print(start,end_)

    roi = img[start:end_,0:img.shape[1]]
    area = (start-end_)*(0-img.shape[1])
    if (area!=0):
        plt.imsave("D:\\Project Himachal\\Data\\Temp\\Temp_roi.png",(roi), cmap=plt.cm.gray )
        fr1 = filled_ratio("D:\\Project Himachal\\Data\\Temp\\Temp_roi.png")
        if (fr1<.95) :
            plt.imsave("D:\\Project Himachal\\Data\\Temp\\Test_MM.png",(roi), cmap=plt.cm.gray )
            makeSameSize("D:\\Project Himachal\\Data\\Temp\\Test_MM.png","D:\\Project Himachal\\Data\\Temp\\Temp_Resized_CharPart.png")
            output = findCharacterCNN("D:\\Project Himachal\\Data\\Temp\\Temp_Resized_CharPart.png","M")
            out.append(output)
    
    if (output==2354):
        margin = 8
    else:
        margin = 3
    
    if(start-margin > 0):
        start_final =start-margin
    else:
        start_final =start
    roi_t = img[0:start_final,0:img.shape[1]]
    area1 = (0-start_final)*(0-img.shape[1])
    if (area1!=0):
        plt.imsave("D:\\Project Himachal\\Data\\Temp\\Temp_roi_t.png",(roi_t), cmap=plt.cm.gray )
        fr2 = filled_ratio("D:\\Project Himachal\\Data\\Temp\\Temp_roi_t.png")
        ratio = (0-img.shape[1])/(start_final-0)
        ht = find_hieght('D:\\Project Himachal\\Data\\Temp\\Temp_roi_t.png')
        if (roi_t.shape[0]*roi_t.shape[1] > 0) and (fr2!=0):
            if (fr2<.95):
                plt.imsave("D:\\Project Himachal\\Data\\Temp\\Test_Top.png",(roi_t), cmap=plt.cm.gray )
                makeSameSize("D:\\Project Himachal\\Data\\Temp\\Test_Top.png","D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png")
                img_clean = removeBottomLine("D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png")
                plt.imsave("D:\\Project Himachal\\Data\\Temp\\Test_Clean.png",(img_clean), cmap=plt.cm.gray )
                if (isEmpty("D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png")!=1):                    
                    output2 = findCharacterCNN("D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png","T")
                    out.append(output2)

    
    if(end_+ 3 < img.shape[0]):
        end_final =end_+ 3
    else:
        end_final =end_        
    roi_b = img[end_final:img.shape[0],0:img.shape[1]]
    area2 = (end_final-img.shape[0])*(0-img.shape[1])
    if (area2!=0) :
        plt.imsave("D:\\Project Himachal\\Data\\Temp\\Temp_roi_b.png",(roi_b), cmap=plt.cm.gray )
        #plt.imsave("D:\\Project Himachal\\Data\\Temp00\\Temp_roi" + str(int(random.random()*100000)) + "_b.png",(roi_b), cmap=plt.cm.gray )
        fr3 = filled_ratio("D:\\Project Himachal\\Data\\Temp\\Temp_roi_b.png")   
        ratio = (0-img.shape[1])/(end_final-img.shape[0])
        ht = find_hieght('D:\\Project Himachal\\Data\\Temp\\Temp_roi_b.png')
        if (roi_b.shape[0]*roi_b.shape[1] > 0) and (fr3!=0):
            if (fr3<.8):
                plt.imsave("D:\\Project Himachal\\Data\\Temp\\Test_Top.png",(roi_t), cmap=plt.cm.gray )
                makeSameSize("D:\\Project Himachal\\Data\\Temp\\Test_Top.png","D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png")
                
                if (isEmpty("D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png")!=1):                    
                    output3 = findCharacterCNN("D:\\Project Himachal\\Data\\Temp\\Temp_Resized_Top.png","B")                    
                    if(str(output3).strip()==str(2352).strip()):
                        out.append(2381)                            
                    if(str(output3).strip()!=str(0).strip()):
                        out.append(output3)
                
                #plt.imsave("D:\\Project Himachal\\Data\\Temp\\B\\" + str(ratio) + "_"  + str(ht) + "_" + str(rand) + "_MM.png",(roi_b), cmap=plt.cm.gray )
                #makeSameSize("D:\\Project Himachal\\Data\\Temp\\B\\" + str(rand) + "_MM.png","D:\\Project Himachal\\Data\\Temp\\B\\" + str(rand) + "_MM.png")


    #print(output)
    
    return out
    
    #plt.imsave("D:\\Project Himachal\\Data\\Temp\\Test_Mt.png",(roi_t), cmap=plt.cm.gray )
    #plt.imsave("D:\\Project Himachal\\Data\\Temp\\Test_Mb.png",(roi_b), cmap=plt.cm.gray )

# Let us now process a single line for complete pipeline of functions

In [11]:
path = "D:\\Project Himachal\\Data\Documents02\\lines\\Page4\\"
lines = []
for (root, dirs, file) in os.walk(path):
    total = len(file)    
    
print(total)

for count in range(total):
    f = path + "Page_04_Line_" + str(count) + ".png"
    print(f)
    try:        
        xx = divide_line_into_words(f)    
        lines.append(xx)
    except Exception as e:
        print("Pareshan" + str(e))

18
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_0.png
1/1 [==============================] - 0s 16ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_1.png
1/1 [==============================] - 0s 3ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_2.png
1/1 [==============================] - 0s 12ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_3.png
1/1 [==============================] - 0s 11ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_4.png
1/1 [==============================] - 0s 11ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_5.png
1/1 [==============================] - 0s 6ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_6.png
1/1 [==============================] - 0s 7ms/step
D:\Project Himachal\Data\Documents02\lines\Page4\Page_04_Line_7.png
1/1 [==============================] - 0s 16ms/step
D:\Project Himachal\Data\Documents02\lin

In [21]:
lines

[[[[2360], [2330, 2366], [2368]],
  [[2354, 2379]],
  [[2332, 2375]],
  [[2361], [2352]],
  [[2311], [2325]],
  [[2350], [2344, 2381, 2352], [2326, 2366]],
  [[2332, 2379]],
  [[2327], [2337, 2366]],
  [[2325], [2352], [2342, 2366], [2368]],
  [[2415]]],
 [[[2361, 2376]],
  [[2360, 2375]],
  [[2332], [2352], [2404], [2340, 2366], [2368]],
  [[2367], [2352, 2367], [2330]],
  [[2312], [2339, 2375]],
  [[2357, 2366], [2354, 2366], [2368]],
  [[2341, 2366], [2368]],
  [[2404]],
  [[2360, 2375]],
  [[2332], [2327], [2340, 2366], [2368]],
  [[2352], [2344], [2352]]],
 [[[2309, 2306], [2342], [2352]],
  [[2367], [2326, 2367], [2309]],
  [[2309], [2340, 2375]],
  [[2332], [2352], [2352], [2404], [2340]],
  [[2313], [2360, 2366], [2368], [2351, 2379], [2312]],
  [[2325], [2331, 2366]],
  [[2352], [2330, 2379], [2352, 2366], [2368]],
  [[2361, 2376]]],
 [[[2309], [2340, 2375]],
  [[2332], [2327], [2340, 2366], [2368]],
  [[2313], [2360]],
  [[2332, 2379]],
  [[2344], [2361, 2366], [2367, 2306]],

In [101]:
out = ""
for lines_ in lines:
    for line in lines_:
        for word in line:
            #print(line)
            line_clean = postProcessWord(line)  
            #print(line_clean)
            word_formed =""
            for char in line_clean:
                word_formed = word_formed + chr(char)
            #print(out)
        out = out + word_formed + " "
    out = out + "\n"
        #print("Final OCR Output")
print(out)

सची लो जे हर इक मन्रखा जो गडा करदी ९ 
है से जगती रिच ईणे वाली थी । से जगती रनर 
अंदर खिअ अते जगत उसीयोई कछा रचोरी है 
अते जगती उस जो नहा पछारणया । से अपणे रनरन 
लोका कफ अया अते अपणेआ तिस जो नही 
कलाग पर जितणेआ उस जो कव्रलार उनहा रन२ 
जो उनी परमेसरे रे पुतर ह्रणे रा इखतयार दिता 
कि से उसेरे ना पर रिवसवास लादे हिन । से जण 
न ता लह कछा अते न सरीरे री इछा कछा अते 
न प्रसे री इछा कछा पर परमेस्रा कछा उतपन 
उु हिन । अते वचने देह धारण कीता अे प४र 
किपा अते सचयाई कने भरपुर होई करी असा 
अ डेरा कीता अते जिदे हारी पिते रे इकलौते री 
वडुयाई असा उसेरी तिदेही ही वड्रयाईं रिदाखी ।। 
य्रहंने उस पर साखी दिती अते हकका पाई रन५ 
करारी गलया जिसेरी वावत मौ गलया थिअ कि 
जे मेरे पिचो इंदा है से मौ कछा उतम है 
इसकरी कि से मौ कछा अगे थिअ से एइयोइ 



In [100]:
jj_ = (lines[2][3])
jj = postProcessWord(lines[2][3])
print(jj,jj_)

for jis in jj_:
    for cho in jis:
        print(chr(cho))

print("***************")
        
for jjj in jj:
    print(chr(jjj))

[2332, 2327, 2340] [[2332], [2352], [2352], [2404], [2340]]
ज
र
र
।
त
***************
ज
ग
त


In [99]:
def postProcessWord(word):
    cleaned = []
    prev_char = -1
    current_char =-1
    next_char=-1
    index = 0
    vovel_list = [2306,2366,2367,2368,2369,2370,2375,2376,2379,2380]
    
    interim = []
    for character in word:
        for idx,char_part in enumerate(character):
            #print(character[-1:],character[:-2])
            if ((character[-1:]==[2367]) and (idx ==(len(character)-1)) and (len(character)>1)):                
                continue            
            else:
                 interim.append(char_part)
    #print(word)
    #print(interim)
    
    for idx,char_part in enumerate(interim):
        #print(cleaned)
        current_char = char_part  
        if (prev_char==2352) and (current_char==2352):   
            continue
        index = index + 1
        #print(int(current_char),int(prev_char))
        if (prev_char==2366) and (current_char==2368):                
            #print(index-2,cleaned)                
            cleaned.pop()
            #cleaned.pop()
            cleaned.insert(index,current_char)            
        elif (prev_char==2352) and (current_char==2404):                
            #print(index-1,cleaned)
            cleaned.pop()
            #cleaned.pop(index-2)
            cleaned.insert(index,2327)   
        elif (prev_char==2309) and (current_char==2366):                
            #print(index-1,cleaned)
            cleaned.pop()
            #cleaned.pop(index-2)
            cleaned.insert(index,2310)              
        elif (prev_char==2367):
            #print("Here",current_char,index)                
            #print(cleaned)
            cleaned.insert(index-2,current_char)  
            index=index+1
            #cleaned.insert(index,2367)               
        else:                
            cleaned.insert(index,current_char)                        

        try:
            if (int(cleaned[-2:-1][0])==2309) and (int(cleaned[-1:][0])==2367):                      
                cleaned.pop()
                cleaned.pop()                
                cleaned.insert(index,2310)  

            if ((int(cleaned[-2:-1][0])) in vovel_list) and ((int(cleaned[-1:][0])) in vovel_list):
                cleaned.pop()                    
        except:
            aa=0



        prev_char = current_char
        #print(cleaned)
    return cleaned

# In case you want to execute on page level only then use this section

In [21]:
xx = divide_page_into_lines('D:\\Project Himachal\\Data\\Documents\\06.jpg')

out = ""
for line in xx:
    for word in line:
        for char in word:
            for cc in char:            
                if((int(cc))==2368):
                    out = out[:-1] + chr(int(cc))
                else:
                    out = out + chr(int(cc))
        out = out + " "
    out = out + "\n" 

print(out)

11

In [4]:
ss = "तीिरिच"
for s in ss:
    print (ord(s),s)

2340 त
2368 ी
2367 ि
2352 र
2367 ि
2330 च


In [5]:
print(chr(2310))

आ


In [8]:
str(int(random.random()*100000))

'61514'